In [38]:
from algo_module import Algo_manager
from datetime import datetime, timedelta
from time import perf_counter
from time import sleep
import json
import os
import warnings
from kiteconnect import KiteConnect
warnings.filterwarnings("ignore")
from broker_module import Broker
import pandas as pd
from IPython.display import display
pd.options.display.max_columns = None

In [2]:
def generate_access_token(broker_secret, request_token) -> str:

        kite = KiteConnect(api_key=broker_secret['kite_api_key'])

        data = kite.generate_session(request_token=request_token, \
            api_secret=broker_secret['kite_api_secret'])
        print("Token Generated")
        del kite 

        return data['access_token']


def generate_algo_manager (**kwargs):

    parent = os.path.dirname(os.getcwd())
    file_path = os.path.join(parent,kwargs['broker_secret_file_name'])

    with open (file_path, "r") as openfile:
        broker_secret = json.load(openfile)

    if not kwargs['is_kite_access_token_available']:
        broker_secret['kite_access_token'] = \
            generate_access_token(broker_secret,kwargs['kite_request_token'])

        with open (file_path, "w") as outfile:
            json.dump(broker_secret,outfile)

    logs_folder_path = kwargs['log_folder_name']
    # logs_folder_path = os.path.join(parent,kwargs['log_folder_name'])

    if kwargs['broker_for_data'].upper() == 'SIM':
        current_datetime = kwargs['day_start_datetime']
    else: 
        current_datetime = datetime.now()

    module_initiation_time = datetime.now()
    algo_manager = Algo_manager(
        broker_for_trade=kwargs['broker_for_trade'],
        broker_for_data=kwargs['broker_for_data'],
        per_trade_fee = kwargs['per_trade_fee'],
        underlying_name=kwargs['underlying_name'],
        kotak_consumer_key=broker_secret['kotak_consumer_key'],
        kotak_access_token=broker_secret['kotak_access_token'],
        kotak_consumer_secret=broker_secret['kotak_consumer_secret'],
        kotak_user_id=broker_secret['kotak_user_id'],
        kotak_access_code=broker_secret['kotak_access_code'],
        kotak_user_password=broker_secret['kotak_user_password'],
        kite_api_key=broker_secret['kite_api_key'],
        kite_access_token=broker_secret['kite_access_token'],
        log_folder=logs_folder_path,
        current_datetime = current_datetime,
        broker_connection_loss = kwargs['broker_connection_loss'],
        exchange_connection_loss = kwargs['exchange_connection_loss'],
        begin_time=kwargs['trading_start_time'],
        close_time=kwargs['trading_close_time'],
        quantity_per_lot = 50,
        lots_traded = kwargs['lots_traded'],
        total_loss_limit_per_lot = -1_500,
        max_trailing_loss_non_expiry_per_lot = -250,
        max_trailing_loss_expiry_per_lot = -200,
        trailing_loss_trigger_per_lot = 1_500,
        non_expiry_day_no_candle_time = kwargs['non_expiry_day_no_candle_time'],
        expiry_day_no_candle_time = kwargs['expiry_day_no_candle_time'],
        candle_length=kwargs['candle_length'],
        historical_data_folder_name = kwargs['historical_data_folder_name'],
        fno_folder_name = kwargs['fno_folder_name'],
        equity_folder_name = kwargs['equity_folder_name']
        )
    print(f'Module Initiation took: {datetime.now()-module_initiation_time}')

    return algo_manager

In [3]:
#For Live Trading
# day_start_datetime = None
# trading_start_time = datetime(2020,1,1,9,58).time()
# trading_close_time = datetime(2020,1,1,15,7).time()
# switch_off_time = datetime(2020,1,1,15,10,0).time()

#For Simulation
# day_start_datetime = datetime(2021,5,17,9,15)
# trading_start_time = datetime(2021,5,17,9,20).time()
# trading_close_time = datetime(2021,5,17,9,25).time()
# switch_off_time =    datetime(2021,5,17,9,25).time()

# For Live Paper trade
day_start_datetime = None
trading_start_time = datetime(2020,1,1,12,0).time()
trading_close_time = datetime(2020,1,1,15,7).time()
switch_off_time =    datetime(2020,1,1,15,10).time()

# non_expiry_day_no_candle_time = datetime(2020, 1, 1, 9, 15).time()
non_expiry_day_no_candle_time = datetime(2020, 1, 1, 14, 30).time()
expiry_day_no_candle_time = datetime(2020, 1, 1, 13, 0).time()

is_kite_access_token_available = False
kite_request_token='tQ30CBaQqWeMfaTddssr2KFXmpssQZSG'

broker_secret_file_name = 'broker_secret.json'

log_folder_name = 'logs'

candle_length = 5
per_trade_fee = -20
lots_traded = 1
underlying_name = 'NIFTY'

broker_for_trade = 'zerodha'
broker_for_data = 'zerodha'

pause_between_iterations = .7 

# broker_connection_loss = [{'start_datetime':datetime(2021,5,17,9,16),
#                             'end_datetime':datetime(2021,5,17,9,18)},
#                         {'start_datetime':datetime(2021,5,17,9,22),
#                         'end_datetime':datetime(2021,5,17,9,23)},
#                         {'start_datetime':datetime(2021,5,17,10,22),
#                         'end_datetime':datetime(2021,5,17,10,23)}]

broker_connection_loss = None
exchange_connection_loss = None

historical_data_folder_name = 'historical data'
fno_folder_name = 'FNO'
equity_folder_name = 'Equity'

algo_manager = generate_algo_manager (day_start_datetime=day_start_datetime,
                    trading_start_time=trading_start_time,
                    trading_close_time = trading_close_time,
                    switch_off_time = switch_off_time,
                    non_expiry_day_no_candle_time = non_expiry_day_no_candle_time,
                    expiry_day_no_candle_time = expiry_day_no_candle_time,
                    is_kite_access_token_available = is_kite_access_token_available,
                    kite_request_token = kite_request_token,
                    broker_secret_file_name = broker_secret_file_name,
                    log_folder_name = log_folder_name,
                    candle_length = candle_length,
                    per_trade_fee = per_trade_fee,
                    lots_traded = lots_traded,
                    broker_for_trade = broker_for_trade,
                    broker_for_data = broker_for_data,
                    underlying_name = underlying_name,
                    pause_between_iterations = pause_between_iterations,
                    historical_data_folder_name = historical_data_folder_name,
                    fno_folder_name = fno_folder_name,
                    equity_folder_name = equity_folder_name,
                    broker_connection_loss = broker_connection_loss,
                    exchange_connection_loss = exchange_connection_loss)

Token Generated
Module Initiation took: 0:00:01.672735


In [93]:
end_time = datetime(2022,2,11,14,38)
current_datetime = datetime.now()

while current_datetime <= end_time:
    initiation_time = perf_counter()
    time_string = current_datetime.strftime('%Y-%b-%d>%I:%M:%S %p, %a                ')
    pnl = algo_manager.broker.get_pnl(initiation_time=initiation_time,current_datetime=current_datetime)
    print(time_string,pnl)
    sleep(.7)
    current_datetime = datetime.now()



In [92]:
end_time = datetime(2022,2,11,15,30)
current_datetime = datetime.now()

while current_datetime <= end_time:
    initiation_time = perf_counter()
    time_string = current_datetime.strftime('%Y-%b-%d>%I:%M:%S %p, %a                ')
    p = pd.DataFrame(algo_manager.broker.kite.positions()['net'])
    p['exchange:instrument_id'] = p['exchange'] + ":" + p['tradingsymbol']
    p['last_price'] = pd.DataFrame.from_dict(algo_manager.broker.kite.ltp(list(p['exchange:instrument_id'])),orient='index').reset_index()['last_price']
    p['qty_in_hand'] = p['buy_quantity'] - p ['sell_quantity']
    p['cash_flow_realized'] = (p['sell_quantity']*p['sell_price']) - (p['buy_quantity'] * p['buy_price'])
    p['cash_flow_unrealized'] = p['qty_in_hand'] * p['last_price']
    total_pnl = p['cash_flow_realized'].sum() + p['cash_flow_unrealized'].sum()
    total_pnl
    print(time_string,total_pnl)
    # sleep(.7)
    current_datetime = datetime.now()


2022-Feb-11>03:17:52 PM, Fri                 -175.0
2022-Feb-11>03:17:52 PM, Fri                 -175.0
2022-Feb-11>03:17:53 PM, Fri                 -175.0
2022-Feb-11>03:17:53 PM, Fri                 -175.0
2022-Feb-11>03:17:53 PM, Fri                 -175.0
2022-Feb-11>03:17:53 PM, Fri                 -175.0
2022-Feb-11>03:17:53 PM, Fri                 -175.0
2022-Feb-11>03:17:54 PM, Fri                 -175.0
2022-Feb-11>03:17:54 PM, Fri                 -175.0
2022-Feb-11>03:17:54 PM, Fri                 -175.0
2022-Feb-11>03:17:54 PM, Fri                 -175.0
2022-Feb-11>03:17:54 PM, Fri                 -175.0
2022-Feb-11>03:17:54 PM, Fri                 -175.0
2022-Feb-11>03:17:55 PM, Fri                 -175.0
2022-Feb-11>03:17:55 PM, Fri                 -175.0
2022-Feb-11>03:17:55 PM, Fri                 -175.0
2022-Feb-11>03:17:55 PM, Fri                 -175.0
2022-Feb-11>03:17:55 PM, Fri                 -175.0
2022-Feb-11>03:17:56 PM, Fri                 -175.0
2022-Feb-11>

In [85]:
p = algo_manager.broker.kite.positions()

In [90]:
pdf = pd.DataFrame(p['day'])
pdf

,tradingsymbol,exchange,instrument_token,product,quantity,overnight_quantity,multiplier,average_price,close_price,last_price,value,pnl,m2m,unrealised,realised,buy_quantity,buy_price,buy_value,buy_m2m,sell_quantity,sell_price,sell_value,sell_m2m,day_buy_quantity,day_buy_price,day_buy_value,day_sell_quantity,day_sell_price,day_sell_value
0,NIFTY2221716600PE,NFO,13526530,MIS,50,0,1,11.2375,0,11.05,-590.0,-37.5,-37.5,-37.5,0,200,11.2375,2247.5,2247.5,150,11.05,1657.5,1657.5,200,11.2375,2247.5,150,11.05,1657.5
1,NIFTY2221716700PE,NFO,13527554,MIS,50,0,1,16.4500,0,14.45,-852.5,-130.0,-130.0,-130.0,0,100,16.4500,1645.0,1645.0,50,15.85,792.5,792.5,100,16.4500,1645.0,50,15.85,792.5
2,NIFTY2221717800CE,NFO,13618946,MIS,50,0,1,19.2000,0,18.15,-960.0,-52.5,-52.5,-52.5,0,50,19.2000,960.0,960.0,0,0.00,0.0,0.0,50,19.2000,960.0,0,0.00,0.0
3,NIFTY2221718100CE,NFO,13644546,MIS,0,0,1,0.0000,0,3.10,-7.5,-7.5,-7.5,-7.5,0,50,3.7000,185.0,185.0,50,3.55,177.5,177.5,50,3.7000,185.0,50,3.55,177.5


In [72]:
p['qty_in_hand'] = p['buy_quantity'] - p ['sell_quantity']
p['cash_flow_realized'] = (p['sell_quantity']*p['sell_price']) - (p['buy_quantity'] * p['buy_price'])
p['cash_flow_unrealized'] = p['qty_in_hand'] * p['last_price']
p['cash_flow_realized'].sum() + p['cash_flow_unrealized'].sum()

-87.5